# 第 7 章 课后习题：数据可视化的综合应用

本笔记本包含四个数据可视化习题的综合实现，涵盖 3D 柱形图、动态波形动画、跨国航线地图和复杂函数 3D 曲面展示。

## 习题 1: 3D 柱形图绘制

### 目标
复现参考图中的 3D 柱状图样式，在 xOy 平面对角线位置生成 4 个柱子，设定随机高度，配置柱子样式、坐标轴标签及标题。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 数据准备：对角位置(1,1),(2,2),(3,3),(4,4)，z轴随机整数(1-10)
np.random.seed(42)  # 设置随机种子以保证可重复性
x_pos = [1, 2, 3, 4]
y_pos = [1, 2, 3, 4]
z_height = np.random.randint(1, 11, size=4)  # 随机高度，高低起伏

# 创建3D图形
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 柱子参数
dx = dy = 0.6  # 柱子的宽度和深度
dz = z_height  # 柱子高度

# 设置渐变色彩
colors = plt.cm.viridis(np.linspace(0.2, 0.9, 4))

# 绘制3D柱状图
for i in range(4):
    ax.bar3d(x_pos[i], y_pos[i], 0, dx, dy, dz[i], 
             color=colors[i], alpha=0.8, edgecolor='black', linewidth=1)

# 添加标签和标题
ax.set_xlabel('X轴', fontsize=12)
ax.set_ylabel('Y轴', fontsize=12)
ax.set_zlabel('Z轴', fontsize=12)
ax.set_title('3D对角柱形图', fontsize=14, fontweight='bold')

# 调整坐标轴范围
ax.set_xlim(0.5, 4.5)
ax.set_ylim(0.5, 4.5)
ax.set_zlim(0, 12)

# 添加网格
ax.grid(True, alpha=0.3)

# 设置视角
ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

print(f"柱子高度: {z_height}")

## 习题 2: 衰减正弦波动画

### 目标
制作波形 "向右推进" 效果的 GIF 动画，基于衰减正弦波方程 $y = e^{-0.1x} \times \sin(x)$，实现相位移动动效。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

# 基础参数设置
x = np.linspace(0, 10*np.pi, 500)  # x轴范围0-10π
fps = 15
total_frames = 50
phi_step = 2*np.pi/total_frames  # 每帧相位移动步长

# 创建画布
fig, ax = plt.subplots(figsize=(12, 6))

# 设置坐标轴和标签
ax.set_xlabel('t', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('衰减正弦波动画', fontsize=14, fontweight='bold')
ax.set_xlim(0, 10*np.pi)
ax.set_ylim(-1.2, 1.2)
ax.grid(True, alpha=0.3)

# 初始化曲线对象
line, = ax.plot([], [], linewidth=2, color='#2E86AB')

# 衰减正弦波方程
def decay_sine(x, phi=0):
    return np.exp(-0.1*x) * np.sin(x + phi)

# 初始化函数
def init():
    line.set_data([], [])
    return line,

# 动画更新函数
def animate(frame):
    phi = frame * phi_step
    y = decay_sine(x, phi)
    line.set_data(x, y)
    return line,

# 创建动画
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=total_frames, interval=1000/fps,
                               blit=True)

# 保存为GIF
writer = PillowWriter(fps=fps)
anim.save('wave_animation.gif', writer=writer)

plt.tight_layout()
plt.show()

print("GIF动画已保存为 wave_animation.gif")

# 显示静态示例
fig_static, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# 原始波形
y1 = decay_sine(x, 0)
ax1.plot(x, y1, linewidth=2, color='#2E86AB')
ax1.set_title('原始衰减正弦波', fontsize=12)
ax1.set_xlabel('t')
ax1.set_ylabel('y')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 10*np.pi)
ax1.set_ylim(-1.2, 1.2)

# 相位偏移后的波形
y2 = decay_sine(x, np.pi/2)
ax2.plot(x, y2, linewidth=2, color='#FF6B6B')
ax2.set_title('相位偏移π/2后的波形', fontsize=12)
ax2.set_xlabel('t')
ax2.set_ylabel('y')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 10*np.pi)
ax2.set_ylim(-1.2, 1.2)

plt.tight_layout()
plt.show()

## 习题 3: 跨国航线地图

### 目标
绘制从北京飞往吉隆坡的航线图，使用圆柱等积投影，标记两城市坐标，绘制海岸线、国家边界及蓝色大圆航线。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.geodesic import Geodesic

# 地理参数设置
lat_range = [-10, 55]  # 纬度范围：南纬10°-北纬55°
lon_range = [60, 150]  # 经度范围：东经60°-东经150°

# 北京和吉隆坡坐标（纬度N为正，经度E为正）
beijing = {'lat': 39.90, 'lon': 116.40, 'name': '北京'}
kualalumpur = {'lat': 3.13, 'lon': 101.68, 'name': '吉隆坡'}

# 创建地图窗口
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(1, 1, 1, 
                     projection=ccrs.PlateCarree())  # 圆柱等积投影

# 设置地图范围
ax.set_extent([lon_range[0], lon_range[1], lat_range[0], lat_range[1]], 
              crs=ccrs.PlateCarree())

# 添加地理特征
ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.8)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.6)
ax.add_feature(cfeature.COASTLINE, linewidth=0.8, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='gray', linestyle='--')
ax.add_feature(cfeature.LAKES, facecolor='lightblue', alpha=0.8)
ax.add_feature(cfeature.RIVERS, linewidth=0.5, edgecolor='blue', alpha=0.6)

# 绘制大圆航线（蓝色，线宽2.5）
geodesic = Geodesic()

# 计算大圆航线上的点
n_points = 100
great_circle_points = geodesic.inverse(
    np.array([[beijing['lon'], beijing['lat']]]),
    np.array([[kualalumpur['lon'], kualalumpur['lat']]]),
    n_points
)

# 提取航线坐标
lons = np.linspace(beijing['lon'], kualalumpur['lon'], n_points)
lats = np.linspace(beijing['lat'], kualalumpur['lat'], n_points)

# 简化的大圆航线绘制
ax.plot([beijing['lon'], kualalumpur['lon']], 
        [beijing['lat'], kualalumpur['lat']], 
        'b-', linewidth=2.5, transform=ccrs.PlateCarree(),
        label='大圆航线')

# 标记城市（红色五角星+中文标签）
ax.plot(beijing['lon'], beijing['lat'], 'r*', markersize=15, 
        transform=ccrs.PlateCarree(), label=beijing['name'])
ax.text(beijing['lon']+2, beijing['lat']+2, beijing['name'], 
        fontsize=12, fontweight='bold', transform=ccrs.PlateCarree())

ax.plot(kualalumpur['lon'], kualalumpur['lat'], 'r*', markersize=15,
        transform=ccrs.PlateCarree(), label=kualalumpur['name'])
ax.text(kualalumpur['lon']+2, kualalumpur['lat']-3, kualalumpur['name'], 
        fontsize=12, fontweight='bold', transform=ccrs.PlateCarree())

# 添加经纬度网格
ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5,
             xlocs=range(lon_range[0], lon_range[1]+1, 10),
             ylocs=range(lat_range[0], lat_range[1]+1, 10))

# 添加标题和图例
ax.set_title('北京-吉隆坡跨国航线图', fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='upper right')

# 计算航线距离
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 地球半径（公里）
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

distance = haversine_distance(beijing['lat'], beijing['lon'], 
                            kualalumpur['lat'], kualalumpur['lon'])

# 添加距离信息文本框
textstr = f'航线距离: {distance:.1f} km'
props = dict(boxstyle='round', facecolor='white', alpha=0.8)
ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=12,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

print(f"从北京到吉隆坡的航线距离约为: {distance:.1f} 公里")

## 习题 4: 复杂函数的 3D 曲面展示

### 目标
绘制经典 Peaks 函数三维曲面，设定 x/y 定义域，配置色彩映射、平滑度，添加颜色条辅助阅读。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

# 数据准备：x/y定义域[-3,3]，高网格密度保证平滑
x = np.linspace(-3, 3, 100)  # 100个采样点，平衡平滑度和性能
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)  # 生成网格

# Peaks函数计算z值
def peaks_function(X, Y):
    """经典Peaks函数"""
    Z = 3 * (1 - X)**2 * np.exp(-(X**2) - (Y + 1)**2) - \
        10 * (X/5 - X**3 - Y**5) * np.exp(-X**2 - Y**2) - \
        1/3 * np.exp(-(X + 1)**2 - Y**2)
    return Z

Z = peaks_function(X, Y)

# 创建3D曲面图
fig = plt.figure(figsize=(14, 10))

# 创建两个子图：一个使用surf，一个使用wireframe
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

# 第一个图：光滑曲面
surf = ax1.plot_surface(X, Y, Z, cmap='viridis', 
                        edgecolor='none', alpha=0.9,
                        linewidth=0, antialiased=True)

# 添加颜色条
fig.colorbar(surf, ax=ax1, shrink=0.5, aspect=5, label='Z值')

# 设置第一个图的标签和标题
ax1.set_xlabel('X轴', fontsize=12)
ax1.set_ylabel('Y轴', fontsize=12)
ax1.set_zlabel('Z轴', fontsize=12)
ax1.set_title('Peaks函数光滑曲面图', fontsize=14, fontweight='bold')

# 调整视角和坐标轴范围
ax1.view_init(elev=30, azim=45)
ax1.set_xlim([-3, 3])
ax1.set_ylim([-3, 3])
ax1.set_zlim([-8, 8])

# 第二个图：网格线曲面
wire = ax2.plot_surface(X, Y, Z, cmap='plasma', 
                        edgecolor='black', alpha=0.8,
                        linewidth=0.2, antialiased=True)

# 添加颜色条
fig.colorbar(wire, ax=ax2, shrink=0.5, aspect=5, label='Z值')

# 设置第二个图的标签和标题
ax2.set_xlabel('X轴', fontsize=12)
ax2.set_ylabel('Y轴', fontsize=12)
ax2.set_zlabel('Z轴', fontsize=12)
ax2.set_title('Peaks函数网格线曲面图', fontsize=14, fontweight='bold')

# 调整第二个图的视角
ax2.view_init(elev=25, azim=60)
ax2.set_xlim([-3, 3])
ax2.set_ylim([-3, 3])
ax2.set_zlim([-8, 8])

# 添加网格
ax1.grid(True, alpha=0.3)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 显示函数的统计信息
print(f"Peaks函数统计信息:")
print(f"Z值范围: [{np.min(Z):.3f}, {np.max(Z):.3f}]")
print(f"Z值平均值: {np.mean(Z):.3f}")
print(f"Z值标准差: {np.std(Z):.3f}")

# 创建等高线图作为补充
fig_contour, (ax3, ax4) = plt.subplots(1, 2, figsize=(14, 6))

# 等高线图
contour = ax3.contour(X, Y, Z, levels=20, cmap='viridis')
ax3.clabel(contour, inline=True, fontsize=8)
ax3.set_title('Peaks函数等高线图', fontsize=12, fontweight='bold')
ax3.set_xlabel('X轴')
ax3.set_ylabel('Y轴')
ax3.set_aspect('equal')
ax3.grid(True, alpha=0.3)

# 填充等高线图
contourf = ax4.contourf(X, Y, Z, levels=20, cmap='viridis')
fig_contour.colorbar(contourf, ax=ax4, label='Z值')
ax4.set_title('Peaks函数填充等高线图', fontsize=12, fontweight='bold')
ax4.set_xlabel('X轴')
ax4.set_ylabel('Y轴')
ax4.set_aspect('equal')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 总结

本笔记本完成了四个数据可视化习题的综合实现：

1. **3D柱形图**: 在对角线位置生成了4个随机高度的柱子，配置了渐变色彩和3D旋转功能
2. **衰减正弦波动画**: 实现了基于$y = e^{-0.1x} \sin(x)$的相位移动动画，保存为GIF格式
3. **跨国航线地图**: 绘制了北京到吉隆坡的大圆航线，包含海岸线、国界线和城市标记
4. **Peaks函数3D曲面**: 展示了经典Peaks函数的三维曲面，配置了多种色彩映射和平滑效果

这些可视化实例涵盖了MATLAB到Python的转换，展示了数据可视化的多种技术和应用场景。